In [1]:
import h5py
import torch
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans

In [ ]:
def kmean(x, k=512):
    x = x.reshape([-1, 2048])
    print("feature sample:", x.shape[0])
    kmeans = MiniBatchKMeans(n_clusters=k, random_state=43, verbose=True).fit(x)
    print("clustering done")
    cluster_centers = kmeans.cluster_centers_
    print("Get centers")
    labels = kmeans.labels_

    cluster_features = [x[labels == i] for i in range(k)]
    cluster_means = [np.mean(cluster, axis=0) for cluster in cluster_features]
    print("Get mean")
    return cluster_centers, cluster_means

In [ ]:
def make_sample(path="data/nextqa/video_feature/ResNet/ResNet_I_train.h5"):
    frame_feats = []
    with h5py.File(path, 'r') as fp:
        vids = fp['vid']
        feat_key = 'ResNet_I'
        feats = fp[feat_key]
        # print(feats.shape) #v_num, clip_num, feat_dim
        for id, (vid, feat) in enumerate(zip(vids, feats)):
            vid = vid.decode("utf-8")
            frame_feats.append(feat)

    frame_feats = np.array(frame_feats)
    k_center, k_mean = kmean(frame_feats)
    k_center = torch.from_numpy(k_center)
    k_mean = torch.tensor(k_mean)
    torch.save({"k_center": k_center, "k_mean": k_mean}, "visual_clusters.npy")

In [8]:
make_sample()

feature sample: 123840


/data2/chenweixing/anaconda3/envs/scannet/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


Init 1/3 with method k-means++
Inertia for init 1/3: 25274.4453125
Init 2/3 with method k-means++
Inertia for init 2/3: 25229.7421875
Init 3/3 with method k-means++
Inertia for init 3/3: 25202.376953125
[MiniBatchKMeans] Reassigning 214 cluster centers.
Minibatch step 1/12093: mean batch inertia: 8.163419723510742
Minibatch step 2/12093: mean batch inertia: 6.471900463104248, ewa inertia: 6.471900463104248
Minibatch step 3/12093: mean batch inertia: 6.063305377960205, ewa inertia: 6.465143389644125
Minibatch step 4/12093: mean batch inertia: 5.910908222198486, ewa inertia: 6.455977817475548
Minibatch step 5/12093: mean batch inertia: 5.856976509094238, ewa inertia: 6.446071932675159
[MiniBatchKMeans] Reassigning 280 cluster centers.
Minibatch step 6/12093: mean batch inertia: 5.842019081115723, ewa inertia: 6.436082508817198
Minibatch step 7/12093: mean batch inertia: 5.932518005371094, ewa inertia: 6.42775489435141
Minibatch step 8/12093: mean batch inertia: 5.739194869995117, ewa ine

/tmp/ipykernel_3132340/1562556451.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449183005/work/torch/csrc/utils/tensor_new.cpp:261.)
  k_mean = torch.tensor(k_mean)
